<a href="https://colab.research.google.com/github/moin-84/Quantum-Chemistry-Simulation/blob/main/ms-H2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip3 install qiskit qiskit-nature pyscf

In [17]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit_algorithms import VQE

In [18]:
driver = PySCFDriver(atom="O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0", basis="6-31G")
es_problem = driver.run()

In [19]:
active_space = ActiveSpaceTransformer(8, 8)
transformed_problem = active_space.transform(es_problem)

In [20]:
# mapper = JordanWignerMapper()
mapper = BravyiKitaevMapper()
qubits = mapper.map(transformed_problem.hamiltonian.second_q_op())
qubits

SparsePauliOp(['IIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIYYX', 'IIIIIIIIIIIIIXYY', 'IIIIIIIIIIXYYIXX', 'IIIIIIIIIIXXYIXY', 'IIIIIIIIIIIIIIZZ', 'IIIIIIIIIIYIYIXZ', 'IIIIIIIIIIXZYIYI', 'IIIIIIIIIYZIYIXZ', 'IIIIIIIIIXZIYIYI', 'IIIIIIIIIIIIIZII', 'IIIIIIIIIIXYYXZI', 'IIIIIIIIIIXXYYZI', 'IIIIIIIIIIIIZZZI', 'IIIIIIIIZIIIXZZI', 'IIIIIIIIIZZIXIII', 'IIIIIIIIIIIZIIII', 'IIIIIIIIIIZZIIII', 'IIIIIIIIIYYZIIII', 'IIIIIIIIIXXIIIII', 'IIIIIIIIIZIIIIII', 'IIIIIIIIZZZIZIII', 'IIIIIIIZIIIIIIII', 'IIIIIYYXIIIIIIII', 'IIIIIXYYIIIIIIII', 'IIXYYIXXIIIIIIII', 'IIXXYIXYIIIIIIII', 'IIIIIIZZIIIIIIII', 'IIYIYIXZIIIIIIII', 'IIXZYIYIIIIIIIII', 'IYZIYIXZIIIIIIII', 'IXZIYIYIIIIIIIII', 'IIIIIZIIIIIIIIII', 'IIXYYXZIIIIIIIII', 'IIXXYYZIIIIIIIII', 'IIIIZZZIIIIIIIII', 'ZIIIXZZIZIIIIIII', 'IZZIXIIIIIIIIIII', 'IIIZIIIIIIIIIIII', 'IIZZIIIIIIIIIIII', 'IYYZIIIIIIIIIIII', 'IXXIIIIIIIIIIIII', 'IZIIIIIIIIIIIIII', 'ZZZIZIIIZIIIIIII', 'IIIIIIIIIIIIIIZI', 'IIIIIIIIIIYIYIXI', 'IIIIIIIIIIXZYIYZ', 'IIIIIIIIIYZIYIXI', 'IIII

In [21]:
qubits.num_qubits

16

In [22]:
# ansatz = TwoLocal(rotation_blocks=["ry", "rx", "rz"], entanglement_blocks="cz", reps=4)
ansatz = EfficientSU2(num_qubits=qubits.num_qubits, reps=4, entanglement="linear")
estimator = Estimator()
optimizer = SPSA(maxiter=100)

<ipython-input-22-d3ce98e05da7>:3: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [23]:
vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(qubits)
result

In [24]:
print(f"Ground State Energy of H2O: {result.eigenvalue.real:.6f} Hartree")

Ground State Energy of H2O: -19.954473 Hartree


In [25]:
## The ground state energy of H2O is approximately -76.063 hartree.